In [309]:
import datetime as dt

import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from torch import Tensor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from torch import nn

In [310]:
raw_data = pd.read_csv('data/train.csv')

In [311]:
numeric_columns = []
numeric_columns.extend(
    list(raw_data.dtypes[raw_data.dtypes == np.int64].index)
)
numeric_columns.extend(
    list(raw_data.dtypes[raw_data.dtypes == np.float64].index)
)

In [312]:
numeric_columns.remove('SalePrice')
numeric_columns.append('SalePrice')

In [313]:
numeric_columns.remove('Id')

In [314]:
numeric_data = pd.DataFrame(raw_data, columns=numeric_columns)

In [315]:
imputer = SimpleImputer(strategy='constant', fill_value=0)
numeric_data[numeric_columns] = imputer.fit_transform(numeric_data[numeric_columns])

In [316]:
numeric_x_columns = list(numeric_data.columns)
numeric_x_columns.remove('SalePrice')
numeric_y_columns = ['SalePrice']

In [317]:
numeric_x_df = pd.DataFrame(numeric_data, columns=numeric_x_columns)
numeric_y_df = pd.DataFrame(numeric_data, columns=numeric_y_columns)

In [318]:
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super().__init__()

        self.linear1 = nn.Linear(D_in, H1)
        self.linear2 = nn.Linear(H1, H2)
        self.linear3 = nn.Linear(H2, H3)
        self.linear4 = nn.Linear(H3, D_out)
    
    def forward(self, x):
        y_pred = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(y_pred).clamp(min=0)
        y_pred = self.linear3(y_pred).clamp(min=0)
        y_pred = self.linear4(y_pred)
        return y_pred

In [319]:
H1, H2, H3 = 500, 1000, 200

In [320]:
scaler = MinMaxScaler()
numeric_data[numeric_columns] = scaler.fit_transform(numeric_data[numeric_columns])

In [321]:
numeric_x_df = pd.DataFrame(numeric_data, columns=numeric_x_columns)
numeric_y_df = pd.DataFrame(numeric_data, columns=numeric_y_columns)

In [322]:
numeric_x = torch.tensor(numeric_x_df.values, dtype=torch.float)
numeric_y = torch.tensor(numeric_y_df.values, dtype=torch.float)

In [323]:
D_in, D_out = numeric_x.shape[1], numeric_y.shape[1]
D_in, D_out

(36, 1)

In [324]:
model2 = Net(D_in, H1, H2, H3, D_out)
loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)

In [325]:
losses2 = []
for t in range(500):
    y_pred = model2(numeric_x)
    loss = loss_fn(y_pred, numeric_y)
    print(t, loss.item())
    losses2.append(loss.item())

    if torch.isnan(loss):
        print('breaking')
        break

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 55.84474182128906
1 18.99766731262207
2 15.503144264221191
3 15.003340721130371
4 14.78355598449707
5 14.592212677001953
6 14.408930778503418
7 14.231032371520996
8 14.05756664276123
9 13.887137413024902
10 13.719549179077148
11 13.555379867553711
12 13.395102500915527
13 13.238471984863281
14 13.085859298706055
15 12.936614990234375
16 12.789073944091797
17 12.643861770629883
18 12.501441955566406
19 12.36193561553955
20 12.225175857543945
21 12.09076976776123
22 11.95872688293457
23 11.828499794006348
24 11.700202941894531
25 11.573634147644043
26 11.448718070983887
27 11.325511932373047
28 11.203658103942871
29 11.082907676696777
30 10.963146209716797
31 10.844878196716309
32 10.727867126464844
33 10.612140655517578
34 10.498095512390137
35 10.38570785522461
36 10.274826049804688
37 10.164780616760254
38 10.05608081817627
39 9.949219703674316
40 9.84399700164795
41 9.740434646606445
42 9.638345718383789
43 9.537242889404297
44 9.43761157989502
45 9.339252471923828
46 9.24199962615

In [326]:
model3 = Net(D_in, H1, H2, H3, D_out)
loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model3.parameters(), lr=1e-4 * 2)

In [327]:
losses3 = []
for t in range(500):
    y_pred = model3(numeric_x)
    loss = loss_fn(y_pred, numeric_y)
    print(t, loss.item())
    losses3.append(loss.item())

    if torch.isnan(loss):
        print('breaking')
        break

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 60.88337326049805
1 18.215024948120117
2 14.999309539794922
3 14.534963607788086
4 14.27204704284668
5 14.020177841186523
6 13.768001556396484
7 13.51701545715332
8 13.267817497253418
9 13.021637916564941
10 12.780128479003906
11 12.54368782043457
12 12.311897277832031
13 12.085285186767578
14 11.863746643066406
15 11.645480155944824
16 11.431645393371582
17 11.220365524291992
18 11.013655662536621
19 10.812431335449219
20 10.615638732910156
21 10.422722816467285
22 10.233092308044434
23 10.046693801879883
24 9.863557815551758
25 9.683391571044922
26 9.50487232208252
27 9.327311515808105
28 9.150994300842285
29 8.97509479522705
30 8.800753593444824
31 8.63440227508545
32 8.474510192871094
33 8.322017669677734
34 8.176645278930664
35 8.03614330291748
36 7.899972438812256
37 7.76770544052124
38 7.639604568481445
39 7.515455722808838
40 7.395398139953613
41 7.279414653778076
42 7.167466163635254
43 7.0589423179626465
44 6.953625679016113
45 6.851871490478516
46 6.753743648529053
47 6.65

In [329]:
model4 = Net(D_in, H1, H2, H3, D_out)
loss_fn = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model4.parameters(), lr=1e-4 * 2)

losses4 = []
for t in range(500):
    y_pred = model4(numeric_x)
    loss = loss_fn(y_pred, numeric_y)
    print(t, loss.item())
    losses3.append(loss.item())

    if torch.isnan(loss):
        print('breaking')
        break

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

0 71.38018798828125
1 45.41987609863281
2 28.138774871826172
3 17.766002655029297
4 13.386710166931152
5 14.001726150512695
6 16.842899322509766
7 18.647140502929688
8 18.373497009277344
9 16.684118270874023
10 14.58810043334961
11 12.78856086730957
12 11.56342601776123
13 10.936119079589844
14 10.78915786743164
15 10.920934677124023
16 11.054197311401367
17 10.984129905700684
18 10.648913383483887
19 10.089993476867676
20 9.407976150512695
21 8.725386619567871
22 8.167917251586914
23 7.810327529907227
24 7.642197132110596
25 7.556824207305908
26 7.398849964141846
27 7.065773963928223
28 6.588794231414795
29 6.10518741607666
30 5.767512798309326
31 5.6366286277771
32 5.603281021118164
33 5.489892482757568
34 5.23457670211792
35 4.937798500061035
36 4.750421524047852
37 4.72040319442749
38 4.733424186706543
39 4.64764404296875
40 4.470881462097168
41 4.3278350830078125
42 4.292874336242676
43 4.29592752456665
44 4.2258734703063965
45 4.094287395477295
46 3.995605707168579
47 3.959897994